### Configurações Pyspark

In [1]:
from pyspark.sql import SparkSession
from pyspark.sql import functions as F
from pyspark.sql import Window

import pandas as pd

def write_to_postgresl(df, tb_name=None, write_mode='None'):
    df.count()
    if tb_name is None:
        raise Exception('Informe o nome da tabela')
    if write_mode is None:
        raise Exception('Informe o mode de escrita: append ou overwrite')
        
    from datetime import datetime
    start_time = datetime.now()
    
    try:
        df.write.jdbc(url, table=tb_name, mode=write_mode, properties=properties)
    except Exception as e:
        print(f'Erro: {e}')
    end_time = datetime.now()
    
    total_time = str(end_time - start_time)
    
    return f'Total time: {total_time} and - Total rows: {df.count()} - Total columns: {len(df.columns)}'



    # Criando a sessão do Spark
spark  = SparkSession.builder \
    .appName("Data Analysis") \
    .config('spark.jars', '/data/IDAF/DATABASECONNECTOR_JAR_FOLDER/postgresql-42.2.18.jar')\
    .config("spark.executor.memory", "8g")\
    .config("spark.driver.memory", "8g")\
    .config("spark.executor.cores", "4")\
    .config("spark.executor.instances","8")\
    .config("spark.sql.shuffle.partitions","96")\
    .config("spark.default.parallelism","96")\
    .getOrCreate()


url = "jdbc:postgresql://localhost:5432/postgres"

properties = {
    "user" : "postgres",
    "password" : "cidacs",
    "driver" : "org.postgresql.Driver"
}

25/04/01 10:52:09 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


In [2]:
from pyspark.sql.types import ArrayType, StringType

def dividir_em_blocos(s):
    import re
    return re.findall('.{1,4}', s)

dividir_udf = F.udf(dividir_em_blocos, ArrayType(StringType()))

### Configurações Pyspark

In [3]:
spark.conf.set("spark.sql.repl.eagerEval.enabled", True)
spark.conf.set("spark.sql.repl.eagerEval.truncate", 100)
spark.conf.set("spark.sql.repl.eagerEval.maxNumRows", 300)

spark.conf.set("spark.sql.debug.maxToStringFields", 100)

pd.set_option("display.max_columns", None) 
pd.set_option("display.max_rows", None)

### Lendo dados enriquecidos

In [4]:
df_input = (spark
            .read
            .parquet('/data/IDAF/PROJETOS/PARCERIA_CIDACS_PHDC/banco_original_enriched')
           )

### Observation

In [5]:
df_observation = (df_input
                  .withColumn('tipobito_sim', F.col('`tipobito_sim.y`'))
                  .withColumn('dtobito_sim', F.to_date(F.col('`dtobito_sim.y`').cast('date')))
#                   .filter(F.col('tipobito_sim').isNotNull()).limit(10)
                  .withColumn('person_id', F.col('person_id_infant'))
                  .withColumn('dtobito_minus_dtnasc', (F.col('dtobito_sim') - F.col('dtnasc_sinasc').cast('date')).cast('int'))
                  .withColumn('tipobito_sim_code', F.when(F.col('tipobito_sim')==1, F.lit(4079844))
                                                    .when((F.col('tipobito_sim')==2) & (F.col('dtobito_minus_dtnasc').between(0,7)), F.lit(4307303))
                                                    .when((F.col('tipobito_sim')==2) & (F.col('dtobito_minus_dtnasc').between(8,28)), F.lit(4173168))
                                                    .when((F.col('tipobito_sim')==2) & (F.col('dtobito_minus_dtnasc')>28), F.lit(4344630))
                                                    .otherwise(None))
                  .withColumn('mesprenat_sinasc_code', F.lit(4311447))
                  .withColumn('consprenat_sinasc_code', F.lit(4313474))
                  .withColumn('observation_concept_id_list', F.concat_ws(',',
                                                                        F.col('tipobito_sim_code'),
                                                                        F.col('mesprenat_sinasc_code'),
                                                                        F.col('consprenat_sinasc_code')))
                  .withColumn('observation_concept_id', F.explode(F.split('observation_concept_id_list', ',')))
                  .withColumn('observation_date', F.when(F.col('observation_concept_id')==4311447, 
                                                         F.add_months(F.col('dtnasc_sinasc').cast('date'), -F.col('mesprenat_sinasc').cast('int')))
                                                         .otherwise(F.col('dtnasc_sinasc')))
                  .withColumn('observation_type_concept_id', F.lit(32879))
                  .withColumn('value_as_number', F.when(F.col('observation_concept_id')==4313474, F.col('consprenat_sinasc')).otherwise(None))
                  .withColumn('observation_id', F.row_number().over(Window.orderBy('person_id', 'dtnasc_sinasc'))+F.lit(1000000000))
                  .withColumn('observation_datetime', F.lit(None))
                  .withColumn('value_as_string', F.lit(None))
                  .withColumn('value_as_concept_id', F.lit(None))
                  .withColumn('qualifier_concept_id', F.lit(None))
                  .withColumn('unit_concept_id', F.lit(None))
                  .withColumn('provider_id', F.lit(None))
                  .withColumn('visit_occurrence_id', F.lit(None))
                  .withColumn('visit_detail_id', F.lit(None))
                  .withColumn('observation_source_value', F.lit(None))
                  .withColumn('observation_source_concept_id', F.lit(None))
                  .withColumn('unit_source_value', F.lit(None))
                  .withColumn('qualifier_source_value', F.lit(None))
                  .withColumn('value_source_value', F.lit(None))
                  .withColumn('observation_event_id', F.lit(None))
                  .withColumn('obs_event_field_concept_id', F.lit(None))
                  .filter(F.col('observation_date').isNotNull())
#                   .filter(F.col('observation_concept_id').isNotNull())
                 ).select(
                    F.col('observation_id').cast('integer'),
                    F.col('person_id').cast('integer'),
                    F.col('observation_concept_id').cast('integer'),
                    F.col('observation_date').cast('date'),
                    F.col('observation_datetime').cast('timestamp'),
                    F.col('observation_type_concept_id').cast('integer'),
                    F.col('value_as_number').cast('float'),
                    F.col('value_as_string').cast('string'),
                    F.col('value_as_concept_id').cast('integer'),
                    F.col('qualifier_concept_id').cast('integer'),
                    F.col('unit_concept_id').cast('integer'),
                    F.col('provider_id').cast('integer'),
                    F.col('visit_occurrence_id').cast('integer'),
                    F.col('visit_detail_id').cast('integer'),
                    F.col('observation_source_value').cast('string'),
                    F.col('observation_source_concept_id').cast('integer'),
                    F.col('unit_source_value').cast('string'),
                    F.col('qualifier_source_value').cast('string'),
                    F.col('value_source_value').cast('string'),
                    F.col('observation_event_id').cast('integer'),
                    F.col('obs_event_field_concept_id').cast('integer')
                    )
                  
                  
df_observation.count()

33201993

## Salvando no Postgresql

In [6]:
write_to_postgresl(df_observation, tb_name='observation', write_mode='append')

25/04/01 10:53:52 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
25/04/01 10:53:52 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
25/04/01 10:53:52 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
25/04/01 10:53:59 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
25/04/01 10:53:59 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.


'Total time: 0:26:57.550314 and - Total rows: 33201993 - Total columns: 21'

## Deprecated

### Salvando CSV 

In [ ]:
# df_observation.repartition(15).write.option("quoteAll",True).csv(f'/data/IDAF/SHARED/omop_tables_revisadas/observation', header=True, mode='overwrite')

## SQL de insertion

In [ ]:
# CREATE TABLE public.drug_exposure (
# 			drug_exposure_id integer NOT NULL,
# 			person_id integer NOT NULL,
# 			drug_concept_id integer NOT NULL,
# 			drug_exposure_start_date date NOT NULL,
# 			drug_exposure_start_datetime TIMESTAMP NULL,
# 			drug_exposure_end_date date NOT NULL,
# 			drug_exposure_end_datetime TIMESTAMP NULL,
# 			verbatim_end_date date NULL,
# 			drug_type_concept_id integer NOT NULL,
# 			stop_reason varchar(20) NULL,
# 			refills integer NULL,
# 			quantity NUMERIC NULL,
# 			days_supply integer NULL,
# 			sig TEXT NULL,
# 			route_concept_id integer NULL,
# 			lot_number varchar(50) NULL,
# 			provider_id integer NULL,
# 			visit_occurrence_id integer NULL,
# 			visit_detail_id integer NULL,
# 			drug_source_value varchar(50) NULL,
# 			drug_source_concept_id integer NULL,
# 			route_source_value varchar(50) NULL,
# 			dose_unit_source_value varchar(50) NULL );



# CREATE TABLE public.drug_exposure_pyspark (
# 			drug_exposure_id varchar,
# 			person_id varchar,
# 			drug_concept_id varchar,
# 			drug_exposure_start_date varchar,
# 			drug_exposure_start_datetime varchar,
# 			drug_exposure_end_date varchar,
# 			drug_exposure_end_datetime varchar,
# 			verbatim_end_date varchar,
# 			drug_type_concept_id varchar,
# 			stop_reason varchar,
# 			refills varchar,
# 			quantity varchar,
# 			days_supply varchar,
# 			sig varchar,
# 			route_concept_id varchar,
# 			lot_number varchar,
# 			provider_id varchar,
# 			visit_occurrence_id varchar,
# 			visit_detail_id varchar,
# 			drug_source_value varchar,
# 			drug_source_concept_id varchar,
# 			route_source_value varchar,
# 			dose_unit_source_value varchar)


# insert into public.drug_exposure (
# drug_exposure_id,
# person_id,
# drug_concept_id,
# drug_exposure_start_date,
# drug_exposure_start_datetime,
# drug_exposure_end_date,
# drug_exposure_end_datetime,
# verbatim_end_date,
# drug_type_concept_id,
# stop_reason,
# refills,
# quantity,
# days_supply,
# sig,
# route_concept_id,
# lot_number,
# provider_id,
# visit_occurrence_id,
# visit_detail_id,
# drug_source_value,
# drug_source_concept_id,
# route_source_value,
# dose_unit_source_value
# )
# SELECT 
# cast(case when drug_exposure_id='' then null else drug_exposure_id end as integer),
# cast(case when person_id='' then null else person_id end as integer),
# cast(case when drug_concept_id='' then null else drug_concept_id end as integer),
# cast(case when drug_exposure_start_date='' then null else drug_exposure_start_date end as date),
# cast(case when drug_exposure_start_datetime='' then null else drug_exposure_start_datetime end as timestamp),
# cast(case when drug_exposure_end_date='' then null else drug_exposure_end_date end as date),
# cast(case when drug_exposure_end_datetime='' then null else drug_exposure_end_datetime end as timestamp),
# cast(case when verbatim_end_date='' then null else verbatim_end_date end as timestamp),
# cast(case when drug_type_concept_id='' then null else drug_type_concept_id end as integer),
# stop_reason,
# cast(case when refills='' then null else refills end as integer),
# cast(case when quantity='' then null else quantity end as decimal),
# cast(case when days_supply='' then null else days_supply end as integer),
# sig,
# cast(case when route_concept_id='' then null else route_concept_id end as integer),
# cast(case when lot_number='' then null else lot_number end as integer),
# cast(case when provider_id='' then null else provider_id end as integer),
# cast(case when visit_occurrence_id='' then null else visit_occurrence_id end as integer),
# cast(case when visit_detail_id='' then null else visit_detail_id end as integer),
# drug_source_value,
# cast(case when drug_source_concept_id='' then null else drug_source_concept_id end as integer),
# route_source_value,
# dose_unit_source_value
# 	FROM public.drug_exposure_pyspark;
